In [ ]:
import numpy as np
import pylab as pl
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
X_train = pd.read_csv('data/X_train_Hi5.csv')

In [ ]:
X_test = pd.read_csv('data/X_test_Hi5.csv')

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train.head()   

In [ ]:
X_test.head()

In [ ]:
print(X_train.dtypes)

In [ ]:
print(X_train.select_dtypes(include='object').nunique())

In [ ]:
is_NaN = X_train.isna()
print(is_NaN.sum())

In [ ]:
y_train = X_train['piezo_groundwater_level_category']
X_train = X_train.drop(['piezo_groundwater_level_category'], axis=1) 

In [ ]:
y_train.head()

In [ ]:
y_train.nunique()

In [ ]:
y_train.unique()

In [ ]:
plt.hist(y_train)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
nan_columns = X_train.isna().sum()
nan_columns = nan_columns[nan_columns > 0]
nan_types = X_train[nan_columns.index].dtypes
nan_summary = pd.DataFrame({
    'Column': nan_columns.index,
    'Number of NaN': nan_columns.values,
    'Type': nan_types.values
})
nan_summary = nan_summary.sort_values(by='Number of NaN', ascending=False)
print(nan_summary)

Premières données d'ID des piezo stations

In [ ]:
# Colonnes à analyser
columns_to_check = [
    'piezo_station_bdlisa_codes',  # Code de balise
    'piezo_station_altitude', 'piezo_station_longitude',
    'piezo_station_latitude', 'piezo_station_bss_code',
    'piezo_station_bss_id', 'piezo_bss_code'
]

# Garder uniquement les colonnes pertinentes
data_to_check = X_train[columns_to_check]

# Calculer le nombre de valeurs uniques pour chaque colonne
unique_counts = data_to_check[columns_to_check].nunique()

# Trouver la colonne avec le plus grand nombre de valeurs uniques
max_unique_col = unique_counts.idxmax()
max_unique_value = unique_counts.max()

print(f"La colonne avec le plus de valeurs uniques est : {max_unique_col} avec {max_unique_value} valeurs uniques.")



In [ ]:
# Vérifier les valeurs manquantes pour les variables choisies
nan_summary = data_to_check.isna().sum()

# Afficher le résumé des NaN par variable
nan_summary = nan_summary[nan_summary > 0]  # Ne garder que les colonnes ayant des NaN
print("Résumé des valeurs manquantes (NaN) pour les variables sélectionnées :")
print(nan_summary)

In [ ]:
# Vérifier si chaque 'piezo_station_bss_code' correspond à un unique 'piezo_station_bdlisa_codes'
subset_data = data_to_check[['piezo_station_bss_code', 'piezo_station_bdlisa_codes']]

# Supprimer les lignes avec des NaN dans les colonnes concernées
subset_data = subset_data.dropna()

# Grouper par 'piezo_station_bss_code' et compter les valeurs uniques dans 'piezo_station_bdlisa_codes'
unique_mapping = subset_data.groupby('piezo_station_bss_code')['piezo_station_bdlisa_codes'].nunique()

# Vérifier si chaque 'piezo_station_bss_code' correspond à un seul 'piezo_station_bdlisa_codes'
ambiguous_codes = unique_mapping[unique_mapping > 1]

if ambiguous_codes.empty:
    print("Chaque 'piezo_station_bss_code' correspond à un unique 'piezo_station_bdlisa_codes'.")
else:
    print("Certains 'piezo_station_bss_code' correspondent à plusieurs 'piezo_station_bdlisa_codes' :")
    print(ambiguous_codes)



--> On utilise piezo_station_bss_code pour remplir les piezo_station_bdlisa_codes.

On se focalise sur les aurtes données des piezo

In [ ]:
# Liste des colonnes à analyser pour les NaN
columns_to_check_nan = [
    'piezo_measurement_date', 
    'piezo_obtention_mode', 'piezo_status', 'piezo_qualification', 
    'piezo_continuity_code', 'piezo_continuity_name', 'piezo_producer_code', 
    'piezo_producer_name', 'piezo_measure_nature_code', 'piezo_measure_nature_name'
]

# Calculer le nombre de NaN pour chaque colonne
nan_summary = X_train[columns_to_check_nan].isna().sum()

# Filtrer les colonnes qui contiennent des NaN
nan_summary = nan_summary[nan_summary > 0]

# Afficher le résultat
print("Colonnes avec des valeurs manquantes (NaN) et leur nombre :")
print(nan_summary)


pour piezo_measure_nature_code : je veux savoir s'il y a une corrélation avec la mesure précédemment réalisée ?

In [ ]:
# Trier les données par 'piezo_station_bss_code' et 'piezo_measurement_date'
if 'piezo_measurement_date' in X_train.columns and 'piezo_station_bss_code' in X_train.columns:
    X_train_sorted = X_train.sort_values(by=['piezo_station_bss_code', 'piezo_measurement_date'])

    # Décaler la colonne 'piezo_measure_nature_code' pour obtenir la valeur précédente par station
    X_train_sorted['previous_measure_nature_code'] = X_train_sorted.groupby('piezo_station_bss_code')['piezo_measure_nature_code'].shift(1)

    # Encodage des colonnes catégoriques si nécessaire
    encoded_data = X_train_sorted[['piezo_measure_nature_code', 'previous_measure_nature_code']].apply(
        lambda col: col.astype('category').cat.codes if col.dtype == 'object' else col
    )

    # Calculer la corrélation entre la mesure actuelle et la mesure précédente
    correlation = encoded_data.corr().iloc[0, 1]
    print(f"Corrélation entre 'piezo_measure_nature_code' et la mesure précédente (par station) : {correlation:.2f}")
else:
    print("Les colonnes 'piezo_measurement_date' et 'piezo_station_bss_code' sont requises pour effectuer ce tri.")


In [ ]:
# Identifier les stations où toutes les valeurs de 'piezo_measure_nature_code' sont NaN
stations_with_all_nan = X_train.groupby('piezo_station_bss_code')['piezo_measure_nature_code'].apply(lambda x: x.isna().all())

# Filtrer pour ne garder que les stations avec uniquement des NaN
stations_with_all_nan = stations_with_all_nan[stations_with_all_nan]

# Afficher les stations concernées
print(f"Nombre de stations avec uniquement des NaN dans 'piezo_measure_nature_code' : {len(stations_with_all_nan)}")
print("Liste des stations concernées :")
print(stations_with_all_nan.index.tolist())


In [ ]:
# Identifier les stations avec des NaN partiels
stations_with_some_nan = X_train.groupby('piezo_station_bss_code')['piezo_measure_nature_code'].apply(
    lambda x: x.isna().any() and not x.isna().all()
)

# Filtrer pour ne garder que les stations correspondantes
stations_with_some_nan = stations_with_some_nan[stations_with_some_nan]

# Affichage des résultats
print(f"Nombre de stations avec des NaN mais pas uniquement des NaN : {len(stations_with_some_nan)}")
print("Liste des stations concernées :")
print(stations_with_some_nan.index.tolist())


In [ ]:
# Trier les données par station et date
X_train_sorted = X_train.sort_values(by=['piezo_station_bss_code', 'piezo_measurement_date']).reset_index(drop=True)

# Calculer les distances entre les indices des valeurs non manquantes
X_train_sorted['non_nan_index'] = X_train_sorted.groupby('piezo_station_bss_code')['piezo_measure_nature_code'].apply(
    lambda x: x.notna().cumsum()
).reset_index(drop=True)

X_train_sorted['distance'] = X_train_sorted.groupby('piezo_station_bss_code')['non_nan_index'].diff()

# Résumé des distances par station
distance_summary = X_train_sorted.groupby('piezo_station_bss_code')['distance'].mean()

# Afficher les résultats
print("Résumé des distances moyennes entre les valeurs non manquantes pour chaque station :")
print(distance_summary)
